[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/H-IAAC/d3vis_ipynb/blob/main/examples/custom_widget.ipynb)

In [ ]:
# !pip install d3vis_ipynb --quiet

In [ ]:
from d3vis_ipynb import CustomWidget, MatrixLayout
import traitlets

# Import Datasets

In [ ]:
import statsmodels.api as sm
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
affairs = sm.datasets.get_rdataset('Affairs', 'AER').data
affairsData = affairs.to_dict(orient='records')
affairs.head()

# CustomWidget

Generates a widget using JavaScript code from a external or internal source. It's an encapsulation of the anywidget library to generate a widget without hanving to implement the render function.

Functions:
- **createWidgetFromUrl**: Creates a widget using a url. Parameters:
    - **paramList**: a list of variables used by the widget
    - **jsUrl**: the url from which the JavaScript code come from

- **createWidgetFromLocalFile**: Creates a widget using a path. Parameters:
    - **paramList**: a list of variables used by the widget
    - **filePath**: the file path from which the JavaScript code come from

- **readFromWeb**: Read a file from web. Parameter:
    - **url**: file's url

CustomWidget rules:
- a class encapluating CustomWidget must be defined with the following variables:
    - **_esm**: must contain the JavaScript code
    - **_css**: must contain the css code (optional)
- variables with the same names as in *paramList* must be defined with traitlets
- extra variables to create callbacks and allow interaction with other widgets

JavaScript file rules:
- must contain a function called `plot` where the widget will be generated with parameters the same as `paramList`
- a few variables are already pre-loaded and can be used inside the file:
    - **d3**: the d3.js library is already loaded
    - **element**: this is the corresponding HTML element where the widget will be built
    - **width**: width of the element
    - **height**: height of the element
    - **model**: connects with the traitlets, used to change the traitlet value

The example bellow shows implementations of widgets using custom JavaScript and CSS files:

## ScatterPlot

In [ ]:
jsUrl = "https://raw.githubusercontent.com/H-IAAC/d3vis_ipynb/main/samples/scatterplot.js"
cssUrl = "https://raw.githubusercontent.com/H-IAAC/d3vis_ipynb/main/samples/scatterplot.css"
jsPath = "../samples/scatterplot.js"
cssPath = "../samples/scatterplot.css"


class CustomScatterplot(CustomWidget):
    # _esm = CustomWidget.createWidgetFromLocalFile(paramList=["data", "x", "y", "hue"], filePath=jsPath)
    # _css = cssPath

    _esm = CustomWidget.createWidgetFromUrl(paramList=["data", "x", "y", "hue"], jsUrl=jsUrl)
    _css = CustomWidget.readFromWeb(cssUrl)
    
    data = traitlets.List([]).tag(sync=True)
    x = traitlets.Unicode().tag(sync=True)
    y = traitlets.Unicode().tag(sync=True)
    hue = traitlets.Unicode().tag(sync=True)
    clickedValue = traitlets.Unicode().tag(sync=True)
    selectedValues = traitlets.List([]).tag(sync=True)

    def on_select_values(self, callback):
        self.observe(callback, names=["selectedValues"])

    def on_click_value(self, callback):
        self.observe(callback, names=["clickedValue"])

scatterplot = CustomScatterplot(data = affairsData, x = 'age', y = 'yearsmarried', hue = "children")

## Barplot

In [ ]:
url = "https://raw.githubusercontent.com/H-IAAC/d3vis_ipynb/main/samples/barplot.js"
path = "../samples/barplot.js"

class CustomBarplot(CustomWidget):
    # _esm = CustomWidget.createWidgetFromLocalFile(paramList=["data", "x", "y", "hue"], filePath=path)
    
    _esm = CustomWidget.createWidgetFromUrl(paramList=["data", "x", "y", "hue"], jsUrl=url)
    
    data = traitlets.List([]).tag(sync=True)
    x = traitlets.Unicode().tag(sync=True)
    y = traitlets.Unicode().tag(sync=True)
    hue = traitlets.Unicode().tag(sync=True)

barplot = CustomBarplot(data=affairsData, x='education', y='rating')

## HistogramPlot

In [ ]:
url = "https://raw.githubusercontent.com/H-IAAC/d3vis_ipynb/main/samples/histogramplot.js"
path = "../samples/histogramplot.js"

class CustomHistogramplot(CustomWidget):
    # _esm = CustomWidget.createWidgetFromLocalFile(paramList=["data", "x", "start", "end", "elementId"], filePath=path)
    
    _esm = CustomWidget.createWidgetFromUrl(paramList=["data", "x", "start", "end", "elementId"], jsUrl=url)
    
    data = traitlets.List([]).tag(sync=True)
    x = traitlets.Unicode().tag(sync=True)
    start = traitlets.Float().tag(sync=True)
    end = traitlets.Float().tag(sync=True)

histplot = CustomHistogramplot(data=affairsData, x='education')

## RangeSlider

In [ ]:
url = "https://raw.githubusercontent.com/H-IAAC/d3vis_ipynb/main/samples/rangeslider.js"
path = "../samples/rangeslider.js"

from d3vis_ipynb import CustomWidget
import traitlets
class CustomRangeSlider(CustomWidget):
    # _esm = CustomWidget.createWidgetFromLocalFile(paramList=["data", "variable", "step", "description"], filePath=path)
    
    _esm = CustomWidget.createWidgetFromUrl(paramList=["data", "variable", "step", "description"], jsUrl=url)
    
    data = traitlets.List([]).tag(sync=True)
    variable = traitlets.Unicode().tag(sync=True)
    step = traitlets.Float().tag(sync=True)
    description = traitlets.Unicode().tag(sync=True)
    minValue = traitlets.Float().tag(sync=True)
    maxValue = traitlets.Float().tag(sync=True)

    def on_drag(self, callback):
        self.observe(callback, names=["minValue", "maxValue"])

rangeSlider = CustomRangeSlider(data=affairsData, variable="age", step=0.1, description="Age:")

## MatrixLayout

In [ ]:
def on_selected(values):
    barplot.data = scatterplot.selectedValues
    histplot.data = scatterplot.selectedValues
    
scatterplot.on_select_values(on_selected)

def on_values_changed(values):
    newData = affairs[affairs["age"].between(
        rangeSlider.minValue, rangeSlider.maxValue)].to_dict(orient='records')
    scatterplot.data = newData

rangeSlider.on_drag(on_values_changed)

matrix = [
    [3, 3, 1, 1], 
    [3, 3, 1, 1],
    [3, 3, 2, 2],
    [4, 4, 2, 2]
]
matrixLayout = MatrixLayout(matrix, style="neumorphism")
matrixLayout.add(barplot, 1)
matrixLayout.add(histplot, 2)
matrixLayout.add(scatterplot, 3)
matrixLayout.add(rangeSlider, 4)

matrixLayout